In [1]:
# Single model seed finder

from tqdm import tqdm
sds = {}
for SEED in tqdm(range(500)):
    #     print(SEED)
    import os
    os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1'
    

    import tensorflow as tf
    tf.compat.v1.logging.set_verbosity(
        0
    )

    import numpy as np
    import random

    def set_seeds(seed=SEED):
        os.environ['PYTHONHASHSEED'] = str(seed)
        random.seed(seed)
        tf.random.set_seed(seed)
        np.random.seed(seed)

    def set_global_determinism(seed=SEED):
        set_seeds(seed=seed)

        os.environ['TF_DETERMINISTIC_OPS'] = '1'
        os.environ['TF_CUDNN_DETERMINISTIC'] = '1'

        tf.config.threading.set_inter_op_parallelism_threads(1)
        tf.config.threading.set_intra_op_parallelism_threads(1)

    # Call the above function with seed value
    set_global_determinism(seed=SEED)

    import glacierml as gl
    import tensorflow as tf
    import numpy as np
    import pandas as pd
    import matplotlib.pyplot as plt
    import imageio
    import math
    from tensorflow import keras
    from tensorflow.keras import layers
    from tensorflow.keras.layers.experimental import preprocessing
    from scipy.stats import gaussian_kde
    import os
    from scipy.stats import norm
    import statsmodels.api as sm
    from scipy.stats import kstest
    from scipy.stats import shapiro 
    import absl.logging
    from tqdm import tqdm
    
    # Set up data
#     n = 0
    df = gl.coregister_data('4')
    df = df.drop('RGIId', axis = 1)
#     df = df.drop(['RGIId','CenLat','CenLon'], axis = 1)
    df_sampler = df.copy()
    df_trainer = df.copy()
#     rs1 = 421
    rs2 = 323
    n = 0
    
    
    # define model parameters
    pr = 0.075

    callback = tf.keras.callbacks.EarlyStopping(
        monitor = 'val_loss',
        min_delta = 0.001,
        patience = 10,
        verbose = 0,
        mode = 'auto',
        baseline = None,
        restore_best_weights = True
    )
    
    trfeat = {}
    trlabs = {}
    tefeat = {}
    telabs = {}
#     normalizer = {}
    
    # split data
#     for n,df in enumerate(df_list):
    (
        trfeat[n], tefeat[n], trlabs[n], telabs[n]
    ) = gl.split_data(df,rs2)
    
    # build 3 models with subset data
    model = {}
    model_history = {}

    l1 = {}
    l2 = {}
#     for n,df in enumerate(df_list):


    total_inputs = (len(df.columns)) * len(df)
    dp = pr * total_inputs
    tp = dp - (len(df.columns) + (len(df.columns)-1) )
    g = (len(df.columns) + (len(df.columns) - 1))
    l2[n] = 4
    l1[n] = int((dp - 1 - g - 2*l2[n]) / (10 + l2[n]))
    normalizer = preprocessing.Normalization(axis=-1)
    normalizer.adapt(np.array(trfeat[n]))

    model[n] = gl.build_dnn_model(
        normalizer, learning_rate = 0.01, 
        layer_1 = l1[n], layer_2 = l2[n],loss = 'mae'
    )

    model_history[n] = model[n].fit(
        trfeat[n],
        trlabs[n],
        validation_split=0.2,
        callbacks = [callback],
        verbose=0, 
        epochs=500
    )
        
        
    y = {}
    y[n] = (model[n].predict(tefeat[n],verbose = 0).flatten())

    perc_res = (y[n] - telabs[n]) / telabs[n]
    mean_perc_res = np.mean(perc_res)
    std_perc_res = np.std(perc_res)
    res = (y[n] - telabs[n])
    mean_res = np.mean(res)
    std_res = np.std(res)
    sds[str(SEED)] = [mean_perc_res,std_perc_res,mean_res,std_res]
df = pd.DataFrame.from_dict(sds,orient = 'index')
import pickle
with open('seeds_4.pkl', 'wb') as file:
    pickle.dump(df, file, protocol=pickle.HIGHEST_PROTOCOL)

  0%|          | 0/500 [00:00<?, ?it/s]2023-09-19 10:09:12.602710: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-09-19 10:09:14.782502: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1956] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
100%|██████████| 500/500 [34:09<00:00,  4.10s/it]


In [ ]:
# # xval seed finder

# from tqdm import tqdm
# sds = {}
# for SEED in tqdm(range(500)):
#     #     print(SEED)
#     import os
#     os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1'
    

#     import tensorflow as tf
#     tf.compat.v1.logging.set_verbosity(
#         0
#     )

#     import numpy as np
#     import random

#     def set_seeds(seed=SEED):
#         os.environ['PYTHONHASHSEED'] = str(seed)
#         random.seed(seed)
#         tf.random.set_seed(seed)
#         np.random.seed(seed)

#     def set_global_determinism(seed=SEED):
#         set_seeds(seed=seed)

#         os.environ['TF_DETERMINISTIC_OPS'] = '1'
#         os.environ['TF_CUDNN_DETERMINISTIC'] = '1'

#         tf.config.threading.set_inter_op_parallelism_threads(1)
#         tf.config.threading.set_intra_op_parallelism_threads(1)

#     # Call the above function with seed value
#     set_global_determinism(seed=SEED)

#     import glacierml as gl
#     import tensorflow as tf
#     import numpy as np
#     import pandas as pd
#     import matplotlib.pyplot as plt
#     import imageio
#     import math
#     from tensorflow import keras
#     from tensorflow.keras import layers
#     from tensorflow.keras.layers.experimental import preprocessing
#     from scipy.stats import gaussian_kde
#     import os
#     from scipy.stats import norm
#     import statsmodels.api as sm
#     from scipy.stats import kstest
#     from scipy.stats import shapiro 
#     import absl.logging
#     from tqdm import tqdm
    
#     # Set up data
# #     n = 0
#     df = gl.coregister_data('4')
# #     df = df.drop('RGIId', axis = 1)
#     df = df.drop(['RGIId','CenLat','CenLon'],axis = 1)
#     df_sampler = df.copy()
#     df_trainer = df.copy()
#     rs1 = 421
#     rs2 = 67

#     df1 = df_sampler.sample(frac = 0.333333333, random_state = rs1)
#     df_sampler = df_sampler.drop(df1.index)

#     df2 = df_sampler.sample(frac = 0.5,random_state = rs1)
#     df_sampler = df_sampler.drop(df2.index)

#     df3 = df_sampler

#     df1test = df_trainer.drop(df1.index)
#     df2test = df_trainer.drop(df2.index)
#     df3test = df_trainer.drop(df3.index)
#     df_list = [df1,df2,df3]

#     df_test_list = [df1test,df2test,df3test]
    
    
    
    
    
    
#     # define model parameters
#     pr = 0.075

#     callback = tf.keras.callbacks.EarlyStopping(
#         monitor = 'val_loss',
#         min_delta = 0.001,
#         patience = 10,
#         verbose = 0,
#         mode = 'auto',
#         baseline = None,
#         restore_best_weights = True
#     )
    
#     trfeat = {}
#     trlabs = {}
#     tefeat = {}
#     telabs = {}
# #     normalizer = {}
    
#     # split data
#     for n,df in enumerate(df_list):
#         (
#             trfeat[n], tefeat[n], trlabs[n], telabs[n]
#         ) = gl.split_data(df,rs2)
    
#     # build 3 models with subset data
#     model = {}
#     model_history = {}

#     l1 = {}
#     l2 = {}
#     for n,df in enumerate(df_list):


#         total_inputs = (len(df.columns)) * len(df)
#         dp = pr * total_inputs
#         tp = dp - (len(df.columns) + (len(df.columns)-1) )
#         g = (len(df.columns) + (len(df.columns) - 1))
#         l2[n] = 4
#         l1[n] = int((dp - 1 - g - 2*l2[n]) / (10 + l2[n]))
#         normalizer = preprocessing.Normalization(axis=-1)
#         normalizer.adapt(np.array(trfeat[n]))

#         model[n] = gl.build_dnn_model(
#             normalizer, learning_rate = 0.01, 
#             layer_1 = l1[n], layer_2 = l2[n],loss = 'mae'
#         )

#         model_history[n] = model[n].fit(
#             trfeat[n],
#             trlabs[n],
#             validation_split=0.2,
#             callbacks = [callback],
#             verbose=0, 
#             epochs=500
#         )
        
        
#     y = {}
#     for n,dftest in enumerate(df_test_list):
#         y[n] = (model[n].predict(dftest.drop('Thickness',axis = 1),verbose = 0).flatten())

    
    
# #     w = {}
#     perc_res = np.array([])
#     res = np.array([])
#     for n,df,dftest in (zip(range(4),df_list,df_test_list)):
#         percent_res = (y[n] - dftest['Thickness']) / dftest['Thickness']
#         residuals = (y[n] - dftest['Thickness'])
#         mean_perc_res = np.mean(perc_res)
# #         mean_est = np.mean(y[n])

# #         bias = mean_perc_res * mean_est

# #         q75 = (np.percentile(percent_res, 75))
# #         q25 = (np.percentile(percent_res, 25))
# #         iqr = q75 - q25
# #         sig = (iqr / 1.34896) * mean_est
# #         w[n] = abs(bias) + sig**2
#         perc_res = np.append(percent_res,perc_res)
#         res = np.append(res,residuals)
#     mean_res = np.mean(res)
#     std_res = np.std(res)
#     mean_perc_res = np.mean(perc_res)
#     std_perc_res = np.std(perc_res)
    

# # #     print(dp)
#     sds[str(SEED)] = [mean_perc_res,std_perc_res,mean_res,std_res]
#     print(sds[str(SEED)])
# import pickle

# df = pd.DataFrame.from_dict(sds,orient = 'index')
# with open('seeds_iterated_xval_locdropped.pkl', 'wb') as file:
#     pickle.dump(df, file, protocol=pickle.HIGHEST_PROTOCOL)
# #     print(sds[str(SEED)])

In [ ]:
# # Kfold seed finder

# from tqdm import tqdm
# sds = {}
# for SEED in tqdm(range(500)):
#     #     print(SEED)
#     import os
#     os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1'
    

#     import tensorflow as tf
#     tf.compat.v1.logging.set_verbosity(
#         0
#     )

#     import numpy as np
#     import random

#     def set_seeds(seed=SEED):
#         os.environ['PYTHONHASHSEED'] = str(seed)
#         random.seed(seed)
#         tf.random.set_seed(seed)
#         np.random.seed(seed)

#     def set_global_determinism(seed=SEED):
#         set_seeds(seed=seed)

#         os.environ['TF_DETERMINISTIC_OPS'] = '1'
#         os.environ['TF_CUDNN_DETERMINISTIC'] = '1'

#         tf.config.threading.set_inter_op_parallelism_threads(1)
#         tf.config.threading.set_intra_op_parallelism_threads(1)

#     # Call the above function with seed value
#     set_global_determinism(seed=SEED)

#     import glacierml as gl
#     import tensorflow as tf
#     import numpy as np
#     import pandas as pd
#     import matplotlib.pyplot as plt
#     import imageio
#     import math
#     from tensorflow import keras
#     from tensorflow.keras import layers
#     from tensorflow.keras.layers.experimental import preprocessing
#     from scipy.stats import gaussian_kde
#     import os
#     from scipy.stats import norm
#     import statsmodels.api as sm
#     from scipy.stats import kstest
#     from scipy.stats import shapiro 
#     import absl.logging
#     from tqdm import tqdm
    
#     # Set up data
# #     n = 0
#     df = gl.coregister_data('4')
# #     df = df.drop('RGIId', axis = 1)
#     df = df.drop(['RGIId','CenLat','CenLon'],axis = 1)
#     df_sampler = df.copy()
#     df_trainer = df.copy()
#     rs1 = 471
#     rs2 = 67

#     df1test = df_sampler.sample(frac = 0.333333333, random_state = rs1)
#     df_sampler = df_sampler.drop(df1test.index)

#     df2test = df_sampler.sample(frac = 0.5,random_state = rs1)
#     df_sampler = df_sampler.drop(df2test.index)

#     df3test = df_sampler

#     df1 = df_trainer.drop(df1test.index)
#     df2 = df_trainer.drop(df2test.index)
#     df3 = df_trainer.drop(df3test.index)
#     df_list = [df1,df2,df3]

#     df_test_list = [df1test,df2test,df3test]
    
    
    
    
    
    
#     # define model parameters
#     pr = 0.075

#     callback = tf.keras.callbacks.EarlyStopping(
#         monitor = 'val_loss',
#         min_delta = 0.001,
#         patience = 10,
#         verbose = 0,
#         mode = 'auto',
#         baseline = None,
#         restore_best_weights = True
#     )
    
#     trfeat = {}
#     trlabs = {}
#     tefeat = {}
#     telabs = {}
# #     normalizer = {}
    
#     # split data
#     for n,df in enumerate(df_list):
#         (
#             trfeat[n], tefeat[n], trlabs[n], telabs[n]
#         ) = gl.split_data(df,rs2)
    
#     # build 3 models with subset data
#     model = {}
#     model_history = {}

#     l1 = {}
#     l2 = {}
#     for n,df in enumerate(df_list):


#         total_inputs = (len(df.columns)) * len(df)
#         dp = pr * total_inputs
#         tp = dp - (len(df.columns) + (len(df.columns)-1) )
#         g = (len(df.columns) + (len(df.columns) - 1))
#         l2[n] = 4
#         l1[n] = int((dp - 1 - g - 2*l2[n]) / (10 + l2[n]))
#         normalizer = preprocessing.Normalization(axis=-1)
#         normalizer.adapt(np.array(trfeat[n]))

#         model[n] = gl.build_dnn_model(
#             normalizer, learning_rate = 0.01, 
#             layer_1 = l1[n], layer_2 = l2[n],loss = 'mae'
#         )

#         model_history[n] = model[n].fit(
#             trfeat[n],
#             trlabs[n],
#             validation_split=0.2,
#             callbacks = [callback],
#             verbose=0, 
#             epochs=500
#         )
        
        
#     y = {}
#     for n,dftest in enumerate(df_test_list):
#         y[n] = (model[n].predict(dftest.drop('Thickness',axis = 1),verbose = 0).flatten())

    
    
#     w = {}
#     perc_res = np.array([])
#     res = np.array([])
#     for n,df,dftest in (zip(range(4),df_list,df_test_list)):
#         percent_res = (y[n] - dftest['Thickness']) / dftest['Thickness']
#         residuals = (y[n] - dftest['Thickness'])
#         mean_perc_res = np.mean(perc_res)
# #         mean_est = np.mean(y[n])

# #         bias = mean_perc_res * mean_est

# #         q75 = (np.percentile(percent_res, 75))
# #         q25 = (np.percentile(percent_res, 25))
# #         iqr = q75 - q25
# #         sig = (iqr / 1.34896) * mean_est
# #         w[n] = abs(bias) + sig**2
#         perc_res = np.append(percent_res,perc_res)
#         res = np.append(res,residuals)
#     mean_res = np.mean(res)
#     std_res = np.std(res)
#     mean_perc_res = np.mean(perc_res)
#     std_perc_res = np.std(perc_res)
    

    
    
#     sds[str(SEED)] = [mean_perc_res,std_perc_res,mean_res,std_res]
# import pickle

# df = pd.DataFrame.from_dict(sds,orient = 'index')
# with open('seeds_iterated_kfold_locdropped.pkl', 'wb') as file:
#     pickle.dump(df, file, protocol=pickle.HIGHEST_PROTOCOL)
# #     print(sds[str(SEED)])

In [ ]:
# import pickle
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_pickle('seeds_iterated_xval.pkl')

df

ilist =[]
for i in range(0,500,75):
    print(i)
    ilist.append(i)
ilist.append(500)

from matplotlib.colors import LinearSegmentedColormap
colors = [(0, 0, 0), (1, 0, 0)] # first color is black, last is red
cm = LinearSegmentedColormap.from_list(
        "Custom", colors, N=20)

plt.scatter(df.index,1-df[0],c = 1-df[0],cmap = cm)
plt.xticks(ilist,ilist)


plt.title('Tensorflow Random Seeds')
plt.ylabel('Probability of Equality of Volume')
plt.xlabel('Random Seed')

In [ ]:


# from tqdm import tqdm
# sds = {}
# for SEED in tqdm(range(500)):
#     #     print(SEED)
#     import os
#     os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1'
    

#     import tensorflow as tf
#     tf.compat.v1.logging.set_verbosity(
#         0
#     )

#     import numpy as np
#     import random

#     def set_seeds(seed=SEED):
#         os.environ['PYTHONHASHSEED'] = str(seed)
#         random.seed(seed)
#         tf.random.set_seed(seed)
#         np.random.seed(seed)

#     def set_global_determinism(seed=SEED):
#         set_seeds(seed=seed)

#         os.environ['TF_DETERMINISTIC_OPS'] = '1'
#         os.environ['TF_CUDNN_DETERMINISTIC'] = '1'

#         tf.config.threading.set_inter_op_parallelism_threads(1)
#         tf.config.threading.set_intra_op_parallelism_threads(1)

#     # Call the above function with seed value
#     set_global_determinism(seed=SEED)

#     import glacierml as gl
#     import tensorflow as tf
#     import numpy as np
#     import pandas as pd
#     import matplotlib.pyplot as plt
#     import imageio
#     import math
#     from tensorflow import keras
#     from tensorflow.keras import layers
#     from tensorflow.keras.layers.experimental import preprocessing
#     from scipy.stats import gaussian_kde
#     import os
#     from scipy.stats import norm
#     import statsmodels.api as sm
#     from scipy.stats import kstest
#     from scipy.stats import shapiro 
#     import absl.logging
#     from tqdm import tqdm
    
#     # Set up data
# #     n = 0
#     df = gl.coregister_data('4')
#     df = df.drop('RGIId', axis = 1)
#     df_sampler = df.copy()
#     df_trainer = df.copy()
#     rs1 = 421
#     rs2 = 67

#     df1 = df_sampler.sample(frac = 0.333333333, random_state = rs1)
#     df_sampler = df_sampler.drop(df1.index)

#     df2 = df_sampler.sample(frac = 0.5,random_state = rs1)
#     df_sampler = df_sampler.drop(df2.index)

#     df3 = df_sampler

#     df1test = df_trainer.drop(df1test.index)
#     df2test = df_trainer.drop(df2test.index)
#     df3test = df_trainer.drop(df3test.index)
#     df_list = [df1,df2,df3]

#     df_test_list = [df1test,df2test,df3test]
    
    
    
    
    
    
#     # define model parameters
#     pr = 0.075

#     callback = tf.keras.callbacks.EarlyStopping(
#         monitor = 'val_loss',
#         min_delta = 0.001,
#         patience = 10,
#         verbose = 0,
#         mode = 'auto',
#         baseline = None,
#         restore_best_weights = True
#     )
    
#     trfeat = {}
#     trlabs = {}
#     tefeat = {}
#     telabs = {}
# #     normalizer = {}
    
#     # split data
#     for n,df in enumerate(df_list):
#         (
#             trfeat[n], tefeat[n], trlabs[n], telabs[n]
#         ) = gl.split_data(df,rs1)
    
#     # build 3 models with subset data
#     model = {}
#     model_history = {}

#     l1 = {}
#     l2 = {}
#     for n,df in enumerate(df_list):


#         total_inputs = (len(df.columns)) * len(df)
#         dp = pr * total_inputs
#         tp = dp - (len(df.columns) + (len(df.columns)-1) )
#         g = (len(df.columns) + (len(df.columns) - 1))
#         l2[n] = 4
#         l1[n] = int((dp - 1 - g - 2*l2[n]) / (10 + l2[n]))
#         normalizer = preprocessing.Normalization(axis=-1)
#         normalizer.adapt(np.array(trfeat[n]))

#         model[n] = gl.build_dnn_model(
#             normalizer, learning_rate = 0.01, 
#             layer_1 = l1[n], layer_2 = l2[n],loss = 'mae'
#         )

#         model_history[n] = model[n].fit(
#             trfeat[n],
#             trlabs[n],
#             validation_split=0.2,
#             callbacks = [callback],
#             verbose=0, 
#             epochs=500
#         )
        
        
#     y = {}
#     for n,dftest in enumerate(df_test_list):
#         y[n] = (model[n].predict(dftest.drop('Thickness',axis = 1),verbose = 0).flatten())

    
    
#     w = {}
#     for n,df,dftest in (zip(range(4),df_list,df_test_list)):
#         perc_res = (y[n] - dftest['Thickness']) / dftest['Thickness']
#         mean_perc_res = np.mean(perc_res)
#         mean_est = np.mean(y[n])

#         bias = mean_perc_res * mean_est

#         q75 = (np.percentile(perc_res, 75))
#         q25 = (np.percentile(perc_res, 25))
#         iqr = q75 - q25
#         sig = (iqr / 1.34896) * mean_est
#         w[n] = abs(bias) + sig**2

    
#     RGI = gl.load_RGI()
#     df = gl.coregister_data('4')
#     rfp = RGI[list(df.drop('Thickness', axis = 1))]
#     est = pd.DataFrame()
#     for n in range(0,3,1):
#         n = str(i)
#     estimates = pd.Series(
#         model[n].predict(rfp.drop('RGIId',axis = 1),verbose = 0).flatten(),name = str(n)
#     )
#     est = pd.concat([est,estimates], axis  = 1)
#     rfp = rfp.join(est)
    
    
#     rfp['w_thickness'] = (rfp['0']/w[0] + rfp['1']/w[1] + rfp['2']/w[2]
#     ) / (1/w[0] + 1/w[1] + 1/w[2])
    
    
#     data = gl.load_notebook_data(coregistration = '4')[['RGIId','FMT']]
#     data = pd.merge(rfp,data,how = 'inner', on = 'RGIId')
#     data = data.dropna(subset = 'FMT')

    
#     data_1 = (data['FMT'] / 1e3) * data['Area']
#     data_2 = (data['w_thickness'] / 1e3) * data['Area']
#     x_1 = np.sort(data_1)
#     y_1 = 1. * np.arange(len(data_1)) / (len(data_1) - 1)
#     x_2 = np.sort(data_2)
#     y_2 = 1. * np.arange(len(data_2)) / (len(data_2) - 1)
    
#     N1 = len(data_1)
#     assert len(data_1) == len(data_2)
#     resample_rate = 0.9
#     Nsample = int(N1 * resample_rate)
#     X = []
#     for i in (range(1000)):
#         sum1 = np.sum(data_1.to_numpy()[np.random.choice(N1, size=Nsample, replace=False)])
#         sum2 = np.sum(data_2.to_numpy()[np.random.choice(N1, size=Nsample, replace=False)])

#         X.append (sum1-sum2)
        
#     t = np.array(X)/resample_rate/1e3
#     mu, std = norm.fit(t) 
#     sig = np.std(X)/resample_rate/1e3
#     value = 0
#     prob1=  norm.cdf(value,loc = mu, scale=sig)
#     prob2= norm.cdf(mu,loc = mu, scale=sig)
#     if prob1 > prob2:
#         p2 = prob2
#         p1 =1 - prob1
#         dp = 1 - (1 - prob1) - prob2
#     elif prob1 < prob2:
#         dp = 1 - (1 - prob2) - prob1
#         p2 = prob1
#         p1 = prob2
# #     print(dp)
#     sds[str(SEED)] = [dp,mu]
# #     print(sds[str(SEED)])